In [ ]:
import pandas as pd
stock_prices = pd.read_csv ('/content/historical_stock_prices.csv')
stock_info = pd.read_csv ('/content/historical_stocks.csv')

In [ ]:
stock_prices.head()

,ticker,open,close,adj_close,low,high,volume,date
0,AHH,11.50,11.58,8.493155,11.25,11.68,4633900.0,2013-05-08
1,AHH,11.66,11.55,8.471151,11.50,11.66,275800.0,2013-05-09
2,AHH,11.55,11.60,8.507822,11.50,11.60,277100.0,2013-05-10
3,AHH,11.63,11.65,8.544494,11.55,11.65,147400.0,2013-05-13
4,AHH,11.60,11.53,8.456484,11.50,11.60,184100.0,2013-05-14


In [ ]:
stock_info.head()

,ticker,exchange,name,sector,industry
0,PIH,NASDAQ,"1347 PROPERTY INSURANCE HOLDINGS, INC.",FINANCE,PROPERTY-CASUALTY INSURERS
1,PIHPP,NASDAQ,"1347 PROPERTY INSURANCE HOLDINGS, INC.",FINANCE,PROPERTY-CASUALTY INSURERS
2,TURN,NASDAQ,180 DEGREE CAPITAL CORP.,FINANCE,FINANCE/INVESTORS SERVICES
3,FLWS,NASDAQ,"1-800 FLOWERS.COM, INC.",CONSUMER SERVICES,OTHER SPECIALTY STORES
4,FCCY,NASDAQ,1ST CONSTITUTION BANCORP (NJ),FINANCE,SAVINGS INSTITUTIONS


In [ ]:
stock_prices['date'] = pd.to_datetime(stock_prices['date'])

In [24]:
#merge
df=pd.merge(stock_prices, stock_info, on ='ticker')

df.set_index('date', inplace= True)

In [25]:
df

,ticker,open,close,adj_close,low,high,volume,exchange,name,sector,industry
date,,,,,,,,,,,
2013-05-08,AHH,11.5000,11.5800,8.493155,11.2500,11.6800,4633900.0,NYSE,"ARMADA HOFFLER PROPERTIES, INC.",FINANCE,REAL ESTATE
2013-05-09,AHH,11.6600,11.5500,8.471151,11.5000,11.6600,275800.0,NYSE,"ARMADA HOFFLER PROPERTIES, INC.",FINANCE,REAL ESTATE
2013-05-10,AHH,11.5500,11.6000,8.507822,11.5000,11.6000,277100.0,NYSE,"ARMADA HOFFLER PROPERTIES, INC.",FINANCE,REAL ESTATE
2013-05-13,AHH,11.6300,11.6500,8.544494,11.5500,11.6500,147400.0,NYSE,"ARMADA HOFFLER PROPERTIES, INC.",FINANCE,REAL ESTATE
2013-05-14,AHH,11.6000,11.5300,8.456484,11.5000,11.6000,184100.0,NYSE,"ARMADA HOFFLER PROPERTIES, INC.",FINANCE,REAL ESTATE
...,...,...,...,...,...,...,...,...,...,...,...
1999-08-17,GTY,14.0625,14.1875,3.500659,14.0625,14.1875,600.0,NYSE,GETTY REALTY CORPORATION,FINANCE,REAL ESTATE
2005-09-01,BDSI,2.3500,2.3400,2.340000,2.2000,2.3500,3700.0,NASDAQ,"BIODELIVERY SCIENCES INTERNATIONAL, INC.",HEALTH CARE,MAJOR PHARMACEUTICALS
2004-12-20,VTNR,43.0000,37.5000,37.500000,37.5000,43.0000,500.0,NASDAQ,"VERTEX ENERGY, INC",ENERGY,INTEGRATED OIL COMPANIES


In [27]:
# create a decade colum
df['decade'] = (df.index.year//10) *10

decade = df.groupby (['decade', 'sector'])

In [28]:
decade

In [29]:
import numpy as np
#dectiong outliers using  IQR

Q1 = df ['close'].quantile(0.25)
Q3 = Q1 = df ['close'].quantile(0.75)

IQR =Q3 -Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound =  Q3 +1.5 * IQR



In [30]:
# correcting illogical values such as negative stock price
df['close'] = df['close'].apply(lambda x: abs(x) if x < 0 else x)


In [31]:
#removing duplicates

df.drop_duplicates(inplace=True)

In [32]:
df['rolling_avg_close'] = df['close'].rolling(window=20).mean()


In [33]:
#volatility (standard deviation over past 30 days)

df['volality'] = df['close'].rolling (window =30).std()

In [34]:
#techical indicator
delta = df['close'].diff()
gain =(delta.where (delta > 0,0)).rolling(window=14).mean()
loss = (-delta.where (delta > 0,0)).rolling(window=14).mean()
rs = gain/loss
df ['RSI'] = 100 - (100/ (1 + rs))

In [35]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [16]:
#standardizing 'close' and 'volume

df[['close', 'volume']] = scaler.fit_transform(df[['close', 'volume']])
df = pd.get_dummies(df, columns=['sector'])


In [36]:
from sklearn.model_selection import train_test_split

In [38]:
#splitting
x_train_val, x_test, y_train_val, y_test = train_test_split(df.drop('close', axis=1), df['close'], test_size=0.2, random_state=2)


In [40]:
#second split to seperate out the validation set
x_train, x_val, y_train, y_val = train_test_split(x_train_val, y_train_val, test_size=0.25, random_state=42)


In [41]:
# Save Clean Data + Splits

df.to_csv('cleaned_data.csv', index=False)
x_train.to_csv('x_train.csv', index=False)
x_val.to_csv('x_val.csv', index=False)
x_test.to_csv('x_test.csv', index=False)
y_train.to_csv('y_train.csv', index=False)
y_val.to_csv('y_val.csv', index=False)
y_test.to_csv('y_test.csv', index=False)

#  Technical Report

** Data Cleaning, Transformation, and Preparation**

The dataset was processed to ensure it was suitable for predictive modeling. The first step involved correcting illogical values. Specifically, negative values in the close price column were converted to their absolute values, as negative stock prices do not exist in real-world financial data. This ensured that all close prices reflected valid, positive values.

Next, numerical features — close and volume — were scaled using StandardScaler, which standardized these columns so that they have a mean of zero and a standard deviation of one. This transformation was applied to ensure that both features contribute proportionally to machine learning models, particularly those sensitive to feature scale, such as regression and distance-based algorithms.

Categorical data in the sector column was converted into numeric format using one-hot encoding. This approach created new binary columns representing each sector without imposing any ordinal relationship between categories. This method was chosen to ensure that machine learning models could interpret sector information appropriately without introducing bias.

The dataset was then split into training, validation, and test sets. First, 80% of the data was reserved for training and validation, while 20% was held out for final testing. The training and validation set was further split, with 75% allocated for training and 25% for validation. This resulted in a final distribution of 60% training data, 20% validation data, and 20% test data. The rationale behind this split was to provide sufficient data for model training, hyperparameter tuning, and unbiased performance evaluation on unseen data.

To validate and document the transformations, the distributions of close and volume were inspected before and after scaling. The transformed features were confirmed to have a mean near zero and a standard deviation close to one, as expected with standard scaling. Additionally, histograms were plotted to visually confirm the changes in distribution.

Finally, the cleaned and transformed dataset was saved as cleaned_data.csv. The feature and target splits for training, validation, and testing were saved as separate CSV files for future use: x_train.csv, x_val.csv, x_test.csv, y_train.csv, y_val.csv, and y_test.csv. A text-based report summarizing the cleaning and preparation process was also saved to ensure reproducibility and transparency of the workflow.

This comprehensive data preparation ensures the dataset is ready for robust and reliable predictive modeling.